In [46]:
import torch
import torch.nn as nn

In [47]:
H=56
W=56
C=64
B=4

In [25]:
# x_list=[]
# in_x1 = torch.rand(B,C,H,W)
# in_x2 = torch.rand(B,C,H,W)
# in_x3 = torch.rand(B,C,H,W)
# in_x4 = torch.rand(B,C,H,W)

# x_list.append(in_x1)
# x_list.append(in_x2)
# x_list.append(in_x3)
# x_list.append(in_x4)
# # print(x_list[0].shape)


In [26]:
# class Conv3d_BN_concat(nn.Module):
#     # input: attention list of length #path
#     def __init__(
#         self,
#         in_ch,
#         out_ch,
#         kernel_size=1,
#         stride=1,
#         pad=0,
#         dilation=1,
#         groups=1,
#         bn_weight_init=1,
#         act_layer=None,
#         norm_cfg="BN",
#     ):
#         super().__init__()
    
#         self.bn = nn.BatchNorm2d(out_ch)

#         torch.nn.init.constant_(self.bn.weight, bn_weight_init)
#         torch.nn.init.constant_(self.bn.bias, 0)
#         for m in self.modules():
#             if isinstance(m, nn.Conv2d):
#                 nn.init.xavier_uniform_(m.weight)
#                 if m.bias is not None:
#                     nn.init.zeros_(m.bias)
       
#         self.interact_concat = nn.Sequential(
#             nn.Conv3d(in_ch, out_ch, kernel_size=(4,1,1)),
#             nn.ReLU()
#         )

#     def forward(self, x_list):
        
#         b,c,h,w = x_list[0].shape
#         print(f"b:{b} c:{c} h:{h} w:{w}")
#         out_3d = []
#         for ip in range(len(x_list)):
# #             print(f'ip:{ip}')
#             in_x = x_list[ip]  
# #             print(f'in each {ip} iteration: {in_x.shape}')
#             in_x = in_x.unsqueeze_(dim=2)
#             out_3d.append(in_x)
# #             print(f'{ip} path extend to shape: {in_x.shape}')

#         x = torch.cat(out_3d, dim=2)
# #         print(f'after concat: {x.shape}')
#         x = torch.squeeze(self.interact_concat(x), dim=2)
# #         print(f'after squeeze: {x.shape}')
#         x = self.bn(x)

#         return x



In [27]:
# conv3d_concat = Conv3d_BN_concat(C,2*C)
# x_out = conv3d_concat(x_list)
# print(x_out.shape)

b:4 c:64 h:56 w:56
torch.Size([4, 128, 56, 56])


https://github.com/VITA-Group/ABD-Net

In [48]:
from torch.nn import Module, Conv2d, Parameter, Softmax

class CAM_Module(Module):
    """ Channel attention module"""

    def __init__(self, in_dim):
        super().__init__()
        self.channel_in = in_dim

        self.gamma = Parameter(torch.zeros(1))
        self.softmax = Softmax(dim=-1)
        self.numpath = 3

    def forward(self, x):
        """
            inputs :
                x : input feature maps( B  C  H  W)
            returns :
                out : attention value + input feature
                attention: B X C X C
        """
        m_batchsize, C,numpath, height, width = x.size()
        proj_query = x.view(m_batchsize, C, numpath, -1) #b c 4 n
        print(f'Shape of query: {proj_query.shape}')
        proj_key = x.view(m_batchsize, C,numpath, -1).permute(0, 1, 3, 2) #b c n 4
        print(f'Shape of key: {proj_key.shape}') 
        energy = torch.matmul(proj_query, proj_key) # b c 4 4
        print(f'Shape of energy: {energy.shape}')
        max_energy_0 = torch.max(energy, -1, keepdim=True)[0].expand_as(energy)
        energy_new = max_energy_0 - energy
        attention = self.softmax(energy_new)
        print(f'Shape of attention: {attention.shape}')
        proj_value = x.view(m_batchsize, C, numpath, -1)
        print(f'Shape of proj_value: {proj_value.shape}')

        out = torch.matmul(attention, proj_value)# can be replace by torch.matmul
        out = out.view(m_batchsize, C, numpath, height, width)

#         logging.debug('cam device: {}, {}'.format(out.device, self.gamma.device))
        gamma = self.gamma.to(out.device)
        out = gamma * out + x
        return out



In [49]:
cam = CAM_Module(in_dim = C)

torch.Size([4, 64, 4, 56, 56])
Shape of query: torch.Size([4, 64, 4, 3136])
Shape of key: torch.Size([4, 64, 3136, 4])
Shape of energy: torch.Size([4, 64, 4, 4])
Shape of attention: torch.Size([4, 64, 4, 4])
Shape of proj_value: torch.Size([4, 64, 4, 3136])
torch.Size([4, 64, 4, 56, 56])


In [50]:
class Conv3d_BN_channel_attention_concat(nn.Module):
    # input: attention list of length #path
    def __init__(
        self,
        in_ch,
        out_ch,
        kernel_size=1,
        stride=1,
        pad=0,
        dilation=1,
        groups=1,
        bn_weight_init=1,
        act_layer=None,
        norm_cfg="BN",
    ):
        super().__init__()
    
        self.bn = nn.BatchNorm2d(out_ch)

        torch.nn.init.constant_(self.bn.weight, bn_weight_init)
        torch.nn.init.constant_(self.bn.bias, 0)
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
       
        self.interact_concat = nn.Sequential(
            nn.Conv3d(in_ch, out_ch, kernel_size=(4,1,1)),
            nn.ReLU()
        )
        self.channelAttention = CAM_Module(in_dim = in_ch)

    def forward(self, x_list):
        
        b,c,h,w = x_list[0].shape
        print(f"b:{b} c:{c} h:{h} w:{w}")
        out_3d = []
        for ip in range(len(x_list)):
#             print(f'ip:{ip}')
            in_x = x_list[ip]  
#             print(f'in each {ip} iteration: {in_x.shape}')
            in_x = in_x.unsqueeze_(dim=2)
            out_3d.append(in_x)
#             print(f'{ip} path extend to shape: {in_x.shape}')

        x = torch.cat(out_3d, dim=2)
        print(f'before channel attention: {x.shape}')
        x = self.channelAttention(x)
        print(f'after concat: {x.shape}')
        x = torch.squeeze(self.interact_concat(x), dim=2)
        print(f'after squeeze: {x.shape}')
        x = self.bn(x)

        return x



In [ ]:
x_list=[]
in_x1 = torch.rand(B,C,H,W)
in_x2 = torch.rand(B,C,H,W)
in_x3 = torch.rand(B,C,H,W)
in_x4 = torch.rand(B,C,H,W)

x_list.append(in_x1)
x_list.append(in_x2)
x_list.append(in_x3)
x_list.append(in_x4)

attention_concat = Conv3d_BN_channel_attention_concat(C,2*C)
x_out = attention_concat(x_list)